In [119]:
import os
import pandas as pd
import numpy as np

In [156]:
df = pd.DataFrame()

In [157]:
column_name = {
    'H02E_H.csv': 'HE051',
    'H00D_H.csv': 'G2041',
    'H04E_H.csv': 'JE051',
    'H06E_H.csv': 'KE051',
    'H08E_H.csv': 'LE051',
    'H10E_H.csv': 'ME051',
    'H12E_H.csv': 'NE051',
    'H98D_H.csv': 'F1825',
    'W2KIDS.csv': 'W8013',
}

index_name = {
    'H02E_H.csv': 6,
    'H00D_H.csv': 5,
    'H04E_H.csv': 7,
    'H06E_H.csv': 8,
    'H08E_H.csv': 9,
    'H10E_H.csv': 10,
    'H12E_H.csv': 11,
    'H98D_H.csv': 4,
    'W2KIDS.csv': 2,
}

In [158]:
tmp = pd.read_csv('../data/H12E_H.csv', sep=';', low_memory=False)
tmp.replace(' ', 0, inplace=True)
tmp['NE051'].value_counts().sum()

14316

In [159]:
tmp.shape

(14316, 155)

In [160]:
answer_map = {
    1: 'yes',
    5: 'no',
    8: '.m',
    9: '.m',
    0: '.m',
    -1: '.m',
}

In [161]:
ids = []
values = []
wave = []

for file in os.listdir('../data'):
    current_df = pd.read_csv(f'../data/{file}', sep=';', low_memory=False)
    current_df.replace(' ', -1, inplace=True)

    try:
        column = column_name[file]

        # print(current_df.groupby('HHID')[column].unique())
        correct_ids = (current_df['HHID'].value_counts() == 1).where(lambda x: x == True).dropna().index
        # correct_ids = child_by_id[child_by_id == 1].index
        # print(correct_ids.shape[0])
        # print(current_df[current_df['HHID'].isin(correct_ids)].shape[0])



        ids = ids + correct_ids.tolist()
        values = values + current_df[current_df['HHID'].isin(correct_ids)][column].values.tolist()
        wave = wave + [index_name[file]] * correct_ids.shape[0]
    except KeyError:
        continue

print(len(ids), len(values))

df['hhidpn'] = ids
df['child'] = values
df['index_wave'] = wave
df.shape    

103430 103430


(103430, 3)

In [162]:
df.head()

hhidpn child  index_wave
0  202408    -1           5
1  202406    -1           5
2  202412    -1           5
3  202405    -1           5
4  202403    -1           5

In [163]:
df['hhidpn'].unique()

array([202408, 202406, 202412, ...,  35843,  38887,  38668], dtype=int64)

In [164]:
df['hhidpn'] = df['hhidpn'].astype(int)
df['child'] = df['child'].astype(int)

In [165]:
df['child'].replace(answer_map, inplace=True)

In [166]:
df['child'].value_counts()

.m     96376
no      3949
yes     3105
Name: child, dtype: int64

In [167]:
df['child'].replace({'yes': '1', 'no': '0'}, inplace=True)

In [168]:
df.to_csv('../child.csv', index=False)